# Import Library

In [76]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
# from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle

## Replace the code below with yours configuration 


In [77]:
# Load data from Google Biquery to local machine

project_id = 'dti-ds'
dataset_id = 'tio_dataset_005'
table_id = 'telco_churn_test'
region = 'us-central1'
bucket_name = 'tio_gcs_005'

# Load Test Data

We would like to load the data from BigQuery 

In [78]:
# load data from BQ
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()

df.head()

/home/tiosyaifuddinlo/miniconda3/envs/telco-churn/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Dependents,tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges
0,Yes,49,No,Yes,DSL,No,No,One year,Yes,51.80
1,No,50,No,Yes,Fiber optic,No,No,One year,Yes,98.25
2,Yes,58,No,No,DSL,No,No,One year,Yes,44.10
3,No,11,No,Yes,DSL,No,No,One year,Yes,61.25
4,Yes,23,Yes,No,Fiber optic,No,No,One year,No,81.85


# Load Model

In [79]:
# Custom transformer
class DropColumns():
    def __init__(self, columns):
        self.columns=columns
    def transform(self, x, y=None):
        return x.drop(self.columns, axis=1)
    def fit(self, x, y=None):
        return self
drop_columns_list = ['one-hot__OnlineBackup_No internet service', 'one-hot__DeviceProtection_No internet service', 
                     'one-hot__TechSupport_No internet service', 'one-hot__DeviceProtection_Yes']

model = pickle.load(open('models/pipeline_model_final.pkl', 'rb'))
model

Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one-hot',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Dependents',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'InternetService',
                                                   'DeviceProtection',
                                                   'TechSupport', 'Contract',
                                                   'PaperlessBilling'])])),
                ('feature selection',
                 <__main__.DropColumns object at 0x7c86a3b9f310>),
                ('resample', RandomUnderSampler(random_state=0)),
                ('scaler', MinMaxScaler()),
                ('model', SGDClassifier(random_state=0))])

# Predict on Loaded Model 

## Real-time prediction

In [80]:
real_time_df = pd.DataFrame(columns=['Dependents','tenure','OnlineSecurity','OnlineBackup','InternetService','DeviceProtection','TechSupport','Contract','PaperlessBilling','MonthlyCharges'])
real_time_df.loc[len(real_time_df)] = ['Yes',	49,	'No', 'Yes', 'DSL',	'No', 'No', 'One year',	'Yes', 51.80]
real_time_df.loc[len(real_time_df)] = ['Yes',	3,	'No', 'Yes', 'DSL',	'No', 'No', 'One year',	'Yes', 51.80]
real_time_df

,Dependents,tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges
0,Yes,49,No,Yes,DSL,No,No,One year,Yes,51.8
1,Yes,3,No,Yes,DSL,No,No,One year,Yes,51.8


In [81]:
model.predict(real_time_df)

array(['No', 'Yes'], dtype='<U3')

## Batch prediction

In [82]:
y_pred = model.predict(df)
df['churn_pred'] = y_pred
df.head()

,Dependents,tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges,churn_pred
0,Yes,49,No,Yes,DSL,No,No,One year,Yes,51.80,No
1,No,50,No,Yes,Fiber optic,No,No,One year,Yes,98.25,No
2,Yes,58,No,No,DSL,No,No,One year,Yes,44.10,No
3,No,11,No,Yes,DSL,No,No,One year,Yes,61.25,No
4,Yes,23,Yes,No,Fiber optic,No,No,One year,No,81.85,No


In [88]:
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id + '_pred')
client.load_table_from_dataframe(df, table_ref).result()

LoadJob<project=dti-ds, location=us-central1, id=78d0e218-a50d-4bc5-b482-38788e51fc09>

## Upload your Model to GCS

In [10]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('models/pipeline_model_final.pkl')
    blob_model.upload_from_filename('models/pipeline_model_final.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


## Model Deployment to Endpoint 

In [11]:
# # run this code to deploy your model, let the code run as default 
# def deploy_model(
#     project: str,
#     location: str,
#     display_name: str,
#     serving_container_image_uri: str,
#     artifact_uri: str,
#     endpoint_name: str,
#     min_replica_count: int = 1,
#     max_replica_count: int = 1,
# ):

#     # Google credentials should be assigned to a service account
#     # Also note that resources should be configured to use the proper, restricted service account

#     aiplatform.init(project=project, location=location)

#     model = aiplatform.Model.upload(
#         display_name=display_name,
#         artifact_uri=artifact_uri,
#         serving_container_image_uri=serving_container_image_uri,
#     )

#     model.wait()

#     # Create an Endpoint
#     endpoint = aiplatform.Endpoint.create(
#         display_name=endpoint_name,
#         project=project,
#         location=location,
#     )

#     # Deploy the Model to the Endpoint
#     endpoint.deploy( 
#         model=model,
#         deployed_model_display_name=display_name,
#         machine_type='e2-standard-2',
#         min_replica_count=min_replica_count,
#         max_replica_count=max_replica_count,
#         sync=True, 
#     )

#     print(model.display_name)
#     print(model.resource_name)
#     return model


In [12]:
# ## model deplyment  
# deploy_model(
#     project=project_id, # Add your project name
#     location=region, # Add your project location
#     display_name='jaya_model_000', # Add display name
#     serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
#     artifact_uri="gs://jaya_gcs_000/model/", # Add directory/folder of model location
#     endpoint_name="jaya-endpoint-000"  # Add endpoint display name
# )


## Model Predictions

In [13]:
# ## predict your data with online prediction here 
# PROJECT_ID = project_id
# # ENDPOINT_ID = "your_endpoint_id"
# # ENDPOINT_ID = '3726357056718897152'
# REGION = region

# aiplatform.init(project=PROJECT_ID, location=REGION)
# endpoint = aiplatform.Endpoint(ENDPOINT_ID)
# prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

# print("PREDICTION:", prediction)

In [ ]:
## in, deploy and test, predict with this template
{
    "instances" : [[50000, 1, 3, 1, 23, 131895, 132154, 129237]]
}